In [1]:
import pandas as pd
import numpy as np

In [2]:
#Before Optimizing the diet plan of the individual, as the constraints varies based on the individual
##Individual must fill-in their informations regarding their body
weight = input("What is your weight, in kilograms: ")
height = input("What is your height: ")
gender = input("What is your gender, man or woman: ")
diabetes = input("Do you have diabetes or risk of diabetes, yes or no: ")
heart_disease = input("Do you risk factors of heart disease, yes or no: ")
lose_weight = input("Do you want to lose weight, yes or no: ")
high_bp = input("Do you have highblood pressure, yes or no: ")
RCI = input("Using the bmi calculator. What is your recommended calorie intake: ")

What is your weight, in kilograms: 65
What is your height: 162
What is your gender, man or woman: woman
Do you have diabetes or risk of diabetes, yes or no: yes
Do you risk factors of heart disease, yes or no: yes
Do you want to lose weight, yes or no: no
Do you have highblood pressure, yes or no: yes
Using the bmi calculator. What is your recommended calorie intake: 1800


In [3]:
# From the form we generate Individual's dataset
individual_data = {'Customer Information':["Weight", "Height","Gender", "Diabetes", "Risk of Heart Disease", "Lose Weight", "High blood Pressure", "Recommended Calorie Intake"],
        'Values': [weight, height, gender, diabetes, heart_disease, lose_weight, high_bp, RCI]}
individual_data = pd.DataFrame(individual_data)
# Convert to use for binary variables
individual_data['Values'].replace('yes', 1, inplace=True)
individual_data['Values'].replace('no', 0, inplace=True)
individual_data['Values'].replace('woman', 1, inplace=True)
individual_data['Values'].replace('man', 0, inplace=True)

# Define dataframe variables in float
weight = float(individual_data['Values'][0])
height = float(individual_data['Values'][1])
gender = float(individual_data['Values'][2])
diabetes = float(individual_data['Values'][3])
heart_disease = float(individual_data['Values'][4])
lose_weight = float(individual_data['Values'][5])
high_bp = float(individual_data['Values'][6])
RCI = float(individual_data['Values'][7])

individual_data


,Customer Information,Values
0,Weight,65
1,Height,162
2,Gender,1
3,Diabetes,1
4,Risk of Heart Disease,1
5,Lose Weight,0
6,High blood Pressure,1
7,Recommended Calorie Intake,1800


In [4]:
# Read csv file
food_data = pd.read_csv('food dataset cleaned.csv', encoding='unicode_escape')

In [5]:
# Filter columns
columns = ['name', 'sugars', 'calories', 'saturated_fat', 'sodium', 'protein', 'carbohydrate', 'cholesterol', 'price']
food_data = food_data[columns]

#Removing g or mg characters
food_data.loc[:, "sugars"] = food_data.loc[:, "sugars"].str.replace("[mg]", "", regex=True)
food_data.loc[:, "saturated_fat"] = food_data.loc[:, "saturated_fat"].str.replace("[mg]", "", regex=True)
food_data.loc[:, "sodium"] = food_data.loc[:, "sodium"].str.replace("[mg]", "", regex=True)
food_data.loc[:, "protein"] = food_data.loc[:, "protein"].str.replace("[mg]", "", regex=True)
food_data.loc[:, "carbohydrate"] = food_data.loc[:, "carbohydrate"].str.replace("[mg]", "", regex=True)
food_data.loc[:, "cholesterol"] = food_data.loc[:, "cholesterol"].str.replace("[mg]", "", regex=True)

#Food prices x 5.30 as food prices is in 100 grams 
# average human eat 3-4 pounds a day = 3.5 pounds in average = 1.167 pounds / meals = 530 grams / meal
food_data["price"] = food_data["price"].apply(lambda x: float(x) *5.30)


In [6]:
# drop some unnecessary food items 
food_data.drop(food_data[food_data.price < 3.0].index, inplace=True)

In [7]:
#Converting string dataframe values into float
food_data["sugars"] = food_data["sugars"].astype(float)
food_data["calories"] = food_data["calories"].astype(float)
food_data["saturated_fat"] = food_data["saturated_fat"].astype(float)
food_data["sodium"] = food_data["sodium"].astype(float)
food_data["protein"] = food_data["protein"].astype(float)
food_data["carbohydrate"] = food_data["carbohydrate"].astype(float)
food_data["cholesterol"] = food_data["cholesterol"].astype(float)
food_data["price"] = food_data["price"].astype(float)

# Converting milligrams values into grams
food_data["sodium"] = food_data["sodium"].apply(lambda x: float(x)/1000)
food_data["cholesterol"] = food_data["cholesterol"].apply(lambda x: float(x)/1000)

#Replace NaN values
food_data.fillna(float(0), inplace = True)

#Test Data
food_data

,name,sugars,calories,saturated_fat,sodium,protein,carbohydrate,cholesterol,price
1,"Pie, lemon, fried pies",0.00,316.0,2.5,0.374,3.00,42.60,0.000,8.45880
2,"Salami, turkey, cooked",1.12,172.0,2.8,1.107,19.20,1.55,0.076,17.62250
3,"McDONALD'S, Hash Brown",0.54,271.0,2.3,0.580,2.48,28.56,0.000,4.22940
4,"Fish, smoked, haddock",0.00,116.0,0.2,0.763,25.23,0.00,0.077,21.14700
6,"Ground turkey, cooked",0.00,203.0,2.7,0.078,27.37,0.00,0.093,10.57350
...,...,...,...,...,...,...,...,...,...
546,"Chicken, original seasoning, rotisserie, cooke...",0.02,233.0,4.1,0.345,22.93,0.02,0.132,5.56871
547,"Beef, braised, cooked, all grades, trimmed to ...",0.00,218.0,3.6,0.070,29.75,0.00,0.093,15.01437
548,"Pork, pan-fried, cooked, separable lean and fa...",0.00,256.0,5.1,0.084,26.81,0.00,0.078,12.40624
549,"Pork, broiled, cooked, separable lean and fat,...",0.00,209.0,3.5,0.055,25.61,0.00,0.084,12.89967


In [8]:
from pulp import *

In [9]:
def Optimization(food_data):
    food_items = list(food_data['name'])
    
    # Create a dictionary of price for all food items
    price = dict(zip(food_items,food_data['price']))

    # Create a dictionary of calories for all food items
    calories = dict(zip(food_items,food_data['calories']))

    # Create a dictionary of protein for all food items
    protein = dict(zip(food_items,food_data['protein']))

    # Create a dictionary of carbohydrate for all food items
    carbohydrate = dict(zip(food_items,food_data['carbohydrate']))

    # Create a dictionary of fats for all food items
    saturated_fat = dict(zip(food_items,food_data['saturated_fat']))

    # Create a dictionary of sodium for all food items
    sodium = dict(zip(food_items,food_data['sodium']))

    # Create a dictionary of cholesterol for all food items
    cholesterol = dict(zip(food_items,food_data['cholesterol']))

    # Create a dictionary of sugars for all food items
    sugars = dict(zip(food_items,food_data['sugars']))

    # Decision variable
    food_vars = LpVariable.dicts("food",food_items,lowBound=0,upBound=1,cat=LpInteger)

    # Define problem
    prob = LpProblem("Simple Diet Problem",LpMinimize)

    # Objective function
    prob += lpSum([price[i] * food_vars[i] for i in food_items])

    # Define Constraints

    # Calories
    prob +=  lpSum([calories[i] * food_vars[i] for i in food_items]) >= 1200.0
    prob += lpSum([calories[i] * food_vars[i] for i in food_items]) <= RCI

    # Protein
    # As it isexpressed in grams per kg of body weight, we multiply by weight (in kg)
    prob += lpSum([protein[i] * food_vars[i] for i in food_items]) * (1.0-lose_weight) <= 0.8 * weight
    prob += lpSum([protein[i] * food_vars[i] for i in food_items]) * lose_weight <= 0.075 * RCI

    # Carbohydrate
    # notes: RCI units in kcal need to convert to grams as carbohydrate dataset in grams
    # 1 kcal = 0.129598 g
    prob += lpSum([carbohydrate[i] * food_vars[i] for i in food_items]) >= 0.45 * RCI * 0.129598
    prob += lpSum([carbohydrate[i] * food_vars[i] for i in food_items]) <= 0.65 * RCI * 0.129598

    # Saturated_fat
    prob += lpSum([saturated_fat[i] * food_vars[i] for i in food_items])* (1.0-gender) <= 30.0 
    prob += lpSum([saturated_fat[i] * food_vars[i] for i in food_items])* gender <= 20.0

    # Sodium 
    prob += lpSum([sodium[i] * food_vars[i] for i in food_items])* (1.0-high_bp) <= 2.3
    prob += lpSum([sodium[i] * food_vars[i] for i in food_items])* high_bp <= 1.5

    # Cholesterol
    prob += lpSum([cholesterol[i] * food_vars[i] for i in food_items])* (1.0-heart_disease) <= 0.3 #if no HD
    prob += lpSum([cholesterol[i] * food_vars[i] for i in food_items])* heart_disease <= 0.2 #if have HD

    # Sugars
    prob += lpSum([sugars[i] * food_vars[i] for i in food_items]) * (1.0-diabetes) <= 50.0 #if no diabetes
    prob += lpSum([sugars[i] * food_vars[i] for i in food_items]) * diabetes <= 25.0 #if diabetes


    #return 3 meals
    prob += lpSum([food_vars[i] for i in food_items]) >= 3.0

    #max and min servings
    for food in food_items:
        prob += food_vars[food] >= 0
        prob += food_vars[food] <= 1

    return food_vars, food_items, prob

In [10]:
## Make a Schedule for a day worth of food
food_vars, food_items, prob = Optimization(food_data)
# Solve the Optimization problem with pulp choice of solver
prob.solve()
## Make a Schedule for a week worth of food
# print the solution
print("Status:", LpStatus[prob.status])
print("Total cost of the diet for the day = $%.2f" % value(prob.objective))
print("The diet consists of:")

# Print the food item
for food in food_items:
    servings = int(food_vars[food].varValue )
    if servings > 0:
        print("- %d serving(s) of %s" % (servings, food))

C:\Users\Fannisa Fahmi\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal
Total cost of the diet for the day = $10.15
The diet consists of:
- 1 serving(s) of Crackers, GOYA CRACKERS, snack
- 1 serving(s) of Snacks, unsalted, wheat-based, sesame sticks
- 1 serving(s) of Fast foods, with condiments and vegetables, regular patty, cheeseburger; single


In [12]:
## Make a Schedule for a week worth of food

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for day in days:
    # Solve the optimization problem for the day
    food_vars, food_items, prob = Optimization(food_data)
    status = prob.solve()
    
    # Print the solution for the day
    print("Status for", day, ":", LpStatus[status])
    print("Total cost of the diet for", day, "= $%.2f" % value(prob.objective))
    print("The diet for", day, "consists of:")
    for food in food_items:
        servings = int(food_vars[food].varValue)
        if servings > 0:
            print("- %d serving(s) of %s" % (servings, food))
            # Remove the food item from food_items
            food_data.drop(food_data[food_data.name == food].index, inplace=True)
    print()  # print a blank line for separation

Status for Monday : Optimal
Total cost of the diet for Monday = $10.15
The diet for Monday consists of:
- 1 serving(s) of Crackers, GOYA CRACKERS, snack
- 1 serving(s) of Snacks, unsalted, wheat-based, sesame sticks
- 1 serving(s) of Fast foods, with condiments and vegetables, regular patty, cheeseburger; single

Status for Tuesday : Optimal
Total cost of the diet for Tuesday = $10.22
The diet for Tuesday consists of:
- 1 serving(s) of SUNSHINE, Original Crackers, CHEEZ-IT
- 1 serving(s) of KEEBLER, Dash of Salt Crackers, CLUB
- 1 serving(s) of Chicken, roasted, cooked, with added solution, meat and skin, thigh, dark meat

Status for Wednesday : Optimal
Total cost of the diet for Wednesday = $10.86
The diet for Wednesday consists of:
- 1 serving(s) of Snacks, plain, bagel chips
- 1 serving(s) of TACO BELL, Soft Taco with steak
- 1 serving(s) of Snacks, salted, made with partially hydrogenated soybean oil, plain, potato chips

Status for Thursday : Optimal
Total cost of the diet for Thu